In [94]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [95]:
spark = (
    SparkSession.builder
    .appName("Curso de PySpark")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .getOrCreate()
)

In [96]:
spark

In [97]:
path = "C:/Users/marlos.barros/Cursos/pyspark_na_pratica/DATASETS/LOGINS.parquet"

In [98]:
df = spark.read.format("parquet").load(path).limit(5)

In [99]:
df.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- email: string (nullable = true)
 |-- senha: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_cadastro: date (nullable = true)
 |-- ipv4: string (nullable = true)
 |-- cor_favorita: string (nullable = true)
 |-- profissao: string (nullable = true)
 |-- telefone: string (nullable = true)



### Agrupamento com Alias
- Função: Agrupa o DataFrame pela coluna **estado**, contando o número de ocorrências de cada estado. A contagem é renomeada como **"qtd"**.

In [100]:
df.groupBy(F.col("estado")).agg(F.count("*").alias("qtd"))

estado,qtd
RR,1
GO,1
MG,1
AC,1
AP,1


### Agrupamento com Alias para Coluna
- Função: Agrupa pela coluna **estado**, mas renomeia essa coluna como **"UF"** no agrupamento. A contagem também é renomeada como **"qtd"**.

In [101]:
df.groupBy(F.col("estado").alias("UF")).agg(F.count("*").alias("qtd"))

UF,qtd
RR,1
GO,1
MG,1
AC,1
AP,1


### Criando Alias para DataFrame
- Função: Cria um **alias** para o DataFrame **df** chamado **"new_df"**, e então seleciona a coluna **cpf** usando o **alias**.

In [102]:
df_1 = df.alias("new_df")
df_1.select("new_df.cpf")

cpf
981.507.362-12
493.705.168-75
398.471.625-73
092.618.354-06
509.427.136-99


### Leitura de Arquivos Parquet com Alias
- Função: Carrega arquivos parquet dos diretórios especificados e cria **alias** para os DataFrames resultantes, renomeando-os como **"livros"** e **"compras"**.

In [103]:
livros = spark.read.parquet("C:/Users/marlos.barros/Cursos/pyspark_na_pratica/DATASETS/LIVROS.parquet").alias("livros")
livros.printSchema()

root
 |-- id: string (nullable = true)
 |-- cnpj_editora: string (nullable = true)
 |-- data_lancamento: date (nullable = true)
 |-- ean: string (nullable = true)
 |-- isbn10: string (nullable = true)
 |-- numero_paginas: string (nullable = true)
 |-- preco: double (nullable = true)



In [104]:
compras = spark.read.parquet("C:/Users/marlos.barros/Cursos/pyspark_na_pratica/DATASETS/COMPRAS.parquet").alias("compras")
compras.printSchema()

root
 |-- id: string (nullable = true)
 |-- cartao_data_expiracao: string (nullable = true)
 |-- cartao_numero: string (nullable = true)
 |-- cartao_bandeira: string (nullable = true)
 |-- cartao_cvc: string (nullable = true)
 |-- codigo_transacao_bancaria: string (nullable = true)
 |-- data: date (nullable = true)
 |-- hora: string (nullable = true)
 |-- ipv4: string (nullable = true)
 |-- ipv6: string (nullable = true)
 |-- cep_entrega: string (nullable = true)
 |-- cd_livro: string (nullable = true)
 |-- cd_cliente: string (nullable = true)



### Join com Alias para DataFrames
- Função: Realiza um **inner join** entre os DataFrames **compras** e **livros**, onde a coluna **cd_livro** de **compras** é igual à coluna **id** de **livros**. O DataFrame resultante é armazenado como **df_2**.

In [105]:
df_2 = compras.join(livros, F.col("cd_livro") == F.col("livros.id"), "inner")
df_2

id,cartao_data_expiracao,cartao_numero,cartao_bandeira,cartao_cvc,codigo_transacao_bancaria,data,hora,ipv4,ipv6,cep_entrega,cd_livro,cd_cliente,id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
012389,11/25,5500804500517692,Discover,959,GB98MPIH622108593...,2021-07-24,03:21:28,62.145.31.164,b23d:58a2:9eff:36...,36629-219,030334762,3339828,030334762,80.463.719/0001-06,2013-05-08,7122725133429,0-85193-340-8,099,123.47
012476,01/31,4609489235873,VISA 16 digit,6979,GB79GAVL233010819...,2021-08-23,11:15:52,185.150.224.52,1e2c:f641:49d1:5a...,17012-747,013721981,7624624,013721981,76.351.940/0001-97,2004-01-27,8786391051693,1-60705-523-6,379,133.43
012478,07/23,30072722359174,American Express,689,GB51SZOU538484531...,2020-01-27,17:08:10,40.179.153.24,9bcd:ee0c:af68:33...,25921298,019458805,8703114,019458805,81.253.490/0001-30,2007-12-13,2628486550239,0-916523-25-X,788,238.54
012534,09/29,4561935154572,VISA 16 digit,353,GB05UNEX021466511...,2021-07-15,07:39:25,86.20.51.194,be57:8f94:132:1cb...,73799370,014347542,7799936,014347542,76.415.890/0001-64,2007-04-05,3883750237349,0-8360-5203-X,452,182.43
012549,01/31,3541220668415122,JCB 15 digit,967,GB97RCCC581942620...,2020-07-22,23:56:14,55.189.220.65,a1fd:9dd:115f:9d5...,50727-454,020215846,6703678,020215846,27.659.813/0001-44,2014-03-25,7684001180416,0-216-16469-9,794,155.18
012574,06/24,4155061214506542,JCB 16 digit,362,GB93YATI197164292...,2020-05-19,10:47:09,212.26.253.42,1815:dc0e:c557:96...,67662-182,010325500,6273720,010325500,27.689.540/0001-80,2000-09-09,8793472196570,0-933627-98-X,306,30.62
012579,01/27,30554213514227,Maestro,9847,GB54PQLN799705859...,2021-04-15,10:36:56,168.197.230.167,7f3b:4ada:4c62:3f...,61193-110,053479015,6977964,053479015,54.327.896/0001-47,2018-12-21,1750535865566,0-518-04786-5,105,207.52
012648,05/24,347386847428278,VISA 16 digit,823,GB90MYZL510193283...,2021-06-19,13:07:48,146.108.131.11,c52d:53d8:371a:34...,10059-723,012331534,6616715,012331534,84.317.925/0001-50,2010-05-29,3177832263150,1-62649-755-9,129,175.67
012675,04/29,4534805013764,Mastercard,384,GB11XMUH824876351...,2021-07-11,07:55:08,148.155.72.234,b290:d2b7:6a58:86...,79292884,035940339,649001,035940339,71.530.496/0001-53,2014-08-11,6764222732753,1-396-90787-4,327,213.6
012698,12/31,4450657280386776,American Express,247,GB79ECRO017868447...,2021-08-11,12:06:48,190.135.5.172,7cbc:4554:7f5d:65...,93609312,058320651,4523531,058320651,82.439.107/0001-03,2013-01-30,6024459409867,1-61357-201-8,290,196.35


### Selecionando Colunas com Alias
- Função: Seleciona todas as colunas do DataFrame **livros** usando o **alias**, juntamente com a coluna **data** do DataFrame **compras**.

In [106]:
df_2.select(F.col("livros.*"), F.col("compras.data"))

id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco,data
030334762,80.463.719/0001-06,2013-05-08,7122725133429,0-85193-340-8,099,123.47,2021-07-24
013721981,76.351.940/0001-97,2004-01-27,8786391051693,1-60705-523-6,379,133.43,2021-08-23
019458805,81.253.490/0001-30,2007-12-13,2628486550239,0-916523-25-X,788,238.54,2020-01-27
014347542,76.415.890/0001-64,2007-04-05,3883750237349,0-8360-5203-X,452,182.43,2021-07-15
020215846,27.659.813/0001-44,2014-03-25,7684001180416,0-216-16469-9,794,155.18,2020-07-22
010325500,27.689.540/0001-80,2000-09-09,8793472196570,0-933627-98-X,306,30.62,2020-05-19
053479015,54.327.896/0001-47,2018-12-21,1750535865566,0-518-04786-5,105,207.52,2021-04-15
012331534,84.317.925/0001-50,2010-05-29,3177832263150,1-62649-755-9,129,175.67,2021-06-19
035940339,71.530.496/0001-53,2014-08-11,6764222732753,1-396-90787-4,327,213.6,2021-07-11
058320651,82.439.107/0001-03,2013-01-30,6024459409867,1-61357-201-8,290,196.35,2021-08-11
